tree census data exploration.  starting with 2015 census

In [19]:
import pandas as pd 
import matplotlib
import plotly as plt

In [5]:
tree_df = pd.read_csv("new_york_tree_census_2015.csv")
tree_df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,2016-06-28,10,0,OnCurb,Alive,Good,Fraxinus pennsylvanica,green ash,...,25,14,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,2015-08-19,9,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,34,13,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,2015-12-30,7,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,32,10,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,2016-07-05,10,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,46,22,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,2015-06-13,4,0,OnCurb,Alive,Good,Prunus virginiana,'Schubert' chokecherry,...,31,10,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017


In [22]:
remove_values = ["Dead", "Stump"]
alive_df = tree_df[~tree_df["status"].isin(remove_values)]
alive_df.describe()

,tree_id,block_id,tree_dbh,stump_diam,zipcode,cb_num,borocode,cncldist,st_assem,st_senate,boro_ct,latitude,longitude,x_sp,y_sp
count,652173.000000,652173.000000,652173.000000,652173.0,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,652173.000000,6.521730e+05,652173.000000,652173.000000,6.521730e+05,652173.000000
mean,365689.637480,313611.213077,11.709485,0.0,10914.407958,343.738388,3.360979,29.992976,50.818103,20.620725,3.407340e+06,40.700991,-73.924587,1.005133e+06,194699.696220
std,208194.926983,114775.077889,8.634239,0.0,656.868397,115.808325,1.167572,14.343667,18.928937,7.378427,1.176612e+06,0.090268,0.123844,3.435720e+04,32886.140551
min,3.000000,100002.000000,0.000000,0.0,83.000000,101.000000,1.000000,1.000000,23.000000,10.000000,1.000201e+06,40.498466,-74.254965,9.133493e+05,120973.792200
25%,186886.000000,221491.000000,5.000000,0.0,10314.000000,303.000000,3.000000,19.000000,33.000000,14.000000,3.012000e+06,40.631725,-73.980973,9.895259e+05,169441.751400
50%,367204.000000,319886.000000,10.000000,0.0,11214.000000,402.000000,4.000000,30.000000,52.000000,21.000000,4.008100e+06,40.700287,-73.913449,1.008235e+06,194444.837500
75%,546632.000000,404665.000000,16.000000,0.0,11365.000000,412.000000,4.000000,43.000000,64.000000,25.000000,4.103300e+06,40.761999,-73.835219,1.029902e+06,216940.389000
max,722694.000000,999999.000000,425.000000,0.0,11697.000000,503.000000,5.000000,51.000000,87.000000,36.000000,5.032300e+06,40.912918,-73.700488,1.067248e+06,271894.092100


In [8]:
filtered_df = alive_df.drop(['created_at', 'stump_diam', 'status', 'user_type', 'sidewalk', 'guards', 'steward', 'st_assem', 
                             'st_senate', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
                             'brch_light', 'brch_shoe', 'brch_other'], axis=1)
filtered_df.head()

Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward',
       'guards', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other', 'address', 'zipcode',
       'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem',
       'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude',
       'longitude', 'x_sp', 'y_sp'],
      dtype='object')

In [6]:
unique_status = tree_df["status"].unique()
unique_status

array(['Alive', 'Dead', 'Stump'], dtype=object)

In [7]:
remove_values = ["Dead", "Stump"]
alive_df = tree_df[~tree_df["status"].isin(remove_values)]
alive_df

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,606945,305778,2016-06-28,10,0,OnCurb,Alive,Good,Fraxinus pennsylvanica,green ash,...,25,14,QN37,Kew Gardens Hills,4125700,New York,40.724339,-73.805180,1.038250e+06,203232.9417
1,160321,341273,2015-08-19,9,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,34,13,QN28,Jackson Heights,4030902,New York,40.756626,-73.894167,1.013571e+06,214953.6472
2,541347,325281,2015-12-30,7,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,32,10,QN76,Baisley Park,4028800,New York,40.679777,-73.788463,1.042923e+06,187008.2671
3,613930,203822,2016-07-05,10,0,OnCurb,Alive,Good,Pyrus calleryana,Callery pear,...,46,22,BK31,Bay Ridge,3005000,New York,40.622743,-74.037543,9.738279e+05,166160.5847
4,18353,338911,2015-06-13,4,0,OnCurb,Alive,Good,Prunus virginiana,'Schubert' chokecherry,...,31,10,QN12,Hammels-Arverne-Edgemere,4095400,New York,40.596514,-73.797622,1.040452e+06,156667.5017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683781,309254,410715,2015-10-11,7,0,OnCurb,Alive,Good,Acer platanoides 'Crimson King',crimson king maple,...,62,24,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5017600,New York,40.528899,-74.168105,9.375181e+05,132013.5133
683782,171664,103174,2015-08-24,12,0,OnCurb,Alive,Good,Tilia americana,American linden,...,74,27,MN28,Lower East Side,1002800,New York,40.727348,-73.976573,9.907431e+05,204270.0951
683783,237788,223344,2015-09-19,2,0,OnCurb,Alive,Poor,Prunus cerasifera,purple-leaf plum,...,51,25,BK33,Carroll Gardens-Columbia Street-Red Hook,3005300,New York,40.672566,-74.011473,9.810674e+05,184310.4162
683786,623784,318368,2016-07-12,18,0,OnCurb,Alive,Good,Quercus rubra,northern red oak,...,31,10,QN55,South Ozone Park,4017800,New York,40.676190,-73.813135,1.036082e+06,185685.7796


In [16]:
#alive_df["user_type"].unique()
#alive_df["guards"].unique()
#alive_df["sidewalk"].unique()
#alive_df["problems"].unique()
#alive_df["steward"].unique()
alive_df.columns


Index(['tree_id', 'block_id', 'created_at', 'tree_dbh', 'stump_diam',
       'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward',
       'guards', 'sidewalk', 'user_type', 'problems', 'root_stone',
       'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other',
       'brch_light', 'brch_shoe', 'brch_other', 'address', 'zipcode',
       'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem',
       'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'latitude',
       'longitude', 'x_sp', 'y_sp'],
      dtype='object')

In [19]:
filtered_df = alive_df.drop(["created_at", "stump_diam", "status", "user_type", "sidewalk", "problems", "guards", "steward", "root_stone", "root_grate", "root_other", "trunk_wire", "trnk_light", "trnk_other", "brch_light", "brch_shoe", "brch_other"])
filtered_df

KeyError: "['created_at', 'stump_diam', 'status', 'user_type', 'sidewalk', 'problems', 'guards', 'steward', 'root_stone', 'root_grate', 'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other'] not found in axis"